# Reverse Geocoder

### Imports

In [1]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

### Reading in the CSV

In [3]:
df = pd.read_csv("../data/fire_archive_M6_157443.csv")
df

,latitude,longitude,brightness,acq_date,confidence,bright_t31,type
0,-15.8931,136.6094,324.4,1/01/2015,28,302.3,0
1,-14.9764,145.2801,320.6,1/01/2015,24,294.8,0
2,-18.5115,139.5995,331.8,1/01/2015,37,305.0,0
3,-17.1160,122.2857,315.7,1/01/2015,0,294.9,0
4,-18.0797,122.6967,313.7,1/01/2015,31,292.9,0
...,...,...,...,...,...,...,...
671992,-22.4415,134.5244,314.9,31/12/2017,88,296.6,0
671993,-22.4800,134.5289,308.8,31/12/2017,69,295.7,0
671994,-22.4323,134.5258,313.8,31/12/2017,86,296.4,0
671995,-22.4707,134.5303,313.1,31/12/2017,83,296.7,0


In [4]:
df["latlong"] = df["latitude"].map(lambda x: str(round(x, 1))) + ", " + df["longitude"].map(lambda x: str(round(x, 1)))
df

,latitude,longitude,brightness,acq_date,confidence,bright_t31,type,latlong
0,-15.8931,136.6094,324.4,1/01/2015,28,302.3,0,"-15.9, 136.6"
1,-14.9764,145.2801,320.6,1/01/2015,24,294.8,0,"-15.0, 145.3"
2,-18.5115,139.5995,331.8,1/01/2015,37,305.0,0,"-18.5, 139.6"
3,-17.1160,122.2857,315.7,1/01/2015,0,294.9,0,"-17.1, 122.3"
4,-18.0797,122.6967,313.7,1/01/2015,31,292.9,0,"-18.1, 122.7"
...,...,...,...,...,...,...,...,...
671992,-22.4415,134.5244,314.9,31/12/2017,88,296.6,0,"-22.4, 134.5"
671993,-22.4800,134.5289,308.8,31/12/2017,69,295.7,0,"-22.5, 134.5"
671994,-22.4323,134.5258,313.8,31/12/2017,86,296.4,0,"-22.4, 134.5"
671995,-22.4707,134.5303,313.1,31/12/2017,83,296.7,0,"-22.5, 134.5"


In [10]:
df_no_dups = df.drop_duplicates(subset=['latlong'])
df_no_dups = df_no_dups.dropna(subset=['latlong'])
df_no_dups

,latitude,longitude,brightness,acq_date,confidence,bright_t31,type,latlong
0,-15.8931,136.6094,324.4,1/01/2015,28,302.3,0,"-15.9, 136.6"
1,-14.9764,145.2801,320.6,1/01/2015,24,294.8,0,"-15.0, 145.3"
2,-18.5115,139.5995,331.8,1/01/2015,37,305.0,0,"-18.5, 139.6"
3,-17.1160,122.2857,315.7,1/01/2015,0,294.9,0,"-17.1, 122.3"
4,-18.0797,122.6967,313.7,1/01/2015,31,292.9,0,"-18.1, 122.7"
...,...,...,...,...,...,...,...,...
671877,-20.6312,119.8370,344.4,31/12/2017,81,322.3,0,"-20.6, 119.8"
671878,-16.7236,129.4765,327.4,31/12/2017,65,295.4,0,"-16.7, 129.5"
671895,-22.4574,134.5212,312.0,31/12/2017,84,297.2,0,"-22.5, 134.5"
671953,-18.3129,134.4838,304.4,31/12/2017,48,277.7,0,"-18.3, 134.5"


### Initializing the reverse geocoder

In [8]:
locator = Nominatim(user_agent="myGeocoder", timeout=10)
rgeocode = RateLimiter(locator.geocode, min_delay_seconds=0.00001)


### Perform the reverse geocoding

In [19]:
print("HI")
print(rgeocode("Cobar, Australia").raw)
print("Done")

SyntaxError: invalid syntax (&lt;ipython-input-19-7078a769ee8a&gt;, line 1)

In [8]:
def get_city_name(x):
    try:
        address = rgeocode(x).raw["address"]
        # print(address)

        if "city" in address:
            return address['city']
        # elif "municipality" in address:
        #     return address['municipality']
        # elif "territory" in address:
        #     return address['territory']
        # elif "suburb" in address:
        #     return address['suburb']
        else:
            return None
    except:
        return None


In [11]:
tqdm_notebook.pandas()

df_small = df_no_dups
df_small['city'] = df_small['latlong'].progress_apply(get_city_name)
df_small

request(&#39;GET&#39;, url, **kwargs)
  File &quot;C:\Users\inos1\anaconda3\envs\3.8.2\lib\site-packages\requests\sessions.py&quot;, line 530, in request
    resp = self.send(prep, **send_kwargs)
  File &quot;C:\Users\inos1\anaconda3\envs\3.8.2\lib\site-packages\requests\sessions.py&quot;, line 643, in send
    r = adapter.send(request, **kwargs)
  File &quot;C:\Users\inos1\anaconda3\envs\3.8.2\lib\site-packages\requests\adapters.py&quot;, line 516, in send
    raise ConnectionError(e, request=request)
requests.exceptions.ConnectionError: HTTPSConnectionPool(host=&#39;nominatim.openstreetmap.org&#39;, port=443): Max retries exceeded with url: /reverse?lat=-32.0&amp;lon=116.0&amp;format=json&amp;addressdetails=1 (Caused by NewConnectionError(&#39;&lt;urllib3.connection.HTTPSConnection object at 0x00000164478B8FD0&gt;: Failed to establish a new connection: [Errno 11001] getaddrinfo failed&#39;))

During handling of the above exception, another exception occurred:

Traceback (most recent 

In [8]:
df_latlong_city_map = df_small.drop(columns=["brightness", "acq_date", "confidence", "bright_t31", "type"])
df_latlong_city_map

,latitude,longitude,latlong,city
0,-15.8931,136.6094,"-16.0, 137.0",Northern Territory
1,-14.9764,145.2801,"-15.0, 145.0",Cook Shire
2,-18.5115,139.5995,"-19.0, 140.0",Cloncurry Shire
3,-17.1160,122.2857,"-17.0, 122.0",None
4,-18.0797,122.6967,"-18.0, 123.0",Shire Of Broome
...,...,...,...,...
649310,-25.4759,140.2284,"-25.0, 140.0",Diamantina Shire
659932,-30.2649,126.1103,"-30.0, 126.0",Shire Of Menzies
661346,-29.4988,131.4937,"-29.0, 131.0",None
661351,-30.1732,132.5465,"-30.0, 133.0",None


In [10]:
df_latlong_city_map.to_csv("../data/map_latlong_city.csv")